# 1. Dataset 살펴보기

In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data_dir = '../../data/MEDICAL-DATASET/Segmentation/'
data_df = pd.read_csv(os.path.join(data_dir,'train.csv'))
data_df.head()

,ImageId,MaskId
0,ID00007637202177411956430_0.jpg,ID00007637202177411956430_mask_0.jpg
1,ID00007637202177411956430_1.jpg,ID00007637202177411956430_mask_1.jpg
2,ID00007637202177411956430_2.jpg,ID00007637202177411956430_mask_2.jpg
3,ID00007637202177411956430_3.jpg,ID00007637202177411956430_mask_3.jpg
4,ID00007637202177411956430_4.jpg,ID00007637202177411956430_mask_4.jpg


In [3]:
# 고객 아이디를 담고있는 컬럼 생성
def customer_id(x):
    return x.split('_')[0]

data_df['ID'] = data_df['ImageId'].apply(customer_id)
data_df.head()

,ImageId,MaskId,ID
0,ID00007637202177411956430_0.jpg,ID00007637202177411956430_mask_0.jpg,ID00007637202177411956430
1,ID00007637202177411956430_1.jpg,ID00007637202177411956430_mask_1.jpg,ID00007637202177411956430
2,ID00007637202177411956430_2.jpg,ID00007637202177411956430_mask_2.jpg,ID00007637202177411956430
3,ID00007637202177411956430_3.jpg,ID00007637202177411956430_mask_3.jpg,ID00007637202177411956430
4,ID00007637202177411956430_4.jpg,ID00007637202177411956430_mask_4.jpg,ID00007637202177411956430


In [24]:
# 고객한명에 대한 데이터만 불러와 동영상으로 확인
def one_customer_data(data_df, index):
    customer = np.unique(data_df['ID'])[index] # 고객 아이디
    img_data_list = list(data_df[data_df['ID']==customer]["ImageId"])
    mask_data_list = list(data_df[data_df['ID']==customer]["MaskId"])
    
    return customer, img_data_list, mask_data_list

index = 50
ID, img_data_list, mask_data_list = one_customer_data(data_df, index)

canvas = np.zeros(shape=(512,512*2+50,3), dtype=np.uint8)

for i in range(len(img_data_list)):
    img = cv2.imread(os.path.join(data_dir,'images' ,img_data_list[i]))
    mask = cv2.imread(os.path.join(data_dir,'masks' ,mask_data_list[i]))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)

    thres = 240
    mask[mask<thres] = 0
    mask[mask>=thres] = 255

    canvas[:, :512, :] = img
    canvas[:,512+50:,:] = mask

    cv2.imshow('image', canvas)
    key = cv2.waitKey(60)
    if key == 27:
        break
    if key == ord('s'):
        cv2.waitKey(0)

cv2.destroyAllWindows()
    

# 2. 데이터셋 및 데이터 로더, transformer 구축

In [4]:
import torch

In [5]:
IMG_SIZE = 224

In [6]:
# 데이터셋
class MY_Dataset():
    def __init__(self, data_dir, phase, transformer=None):
        self.phase = phase
        self.img_dir = os.path.join(data_dir,phase,'images')
        self.mask_dir = os.path.join(data_dir,phase,'masks')
        self.img_file_list = [img for img in os.listdir(self.img_dir) if img.endswith('jpg')]
        self.mask_file_list = [mask for mask in os.listdir(self.mask_dir) if mask.endswith('jpg')]
        assert len(self.img_file_list) == len(self.mask_file_list) # 두 데이터의 길이가 다르면 오류

        self.transformer= transformer


    def __len__(self):
        return len(self.img_file_list)


    def __getitem__(self, index):
        img = cv2.imread(os.path.join(self.img_dir,self.img_file_list[index]))
        img = cv2.resize(img, dsize=(IMG_SIZE,IMG_SIZE), interpolation=cv2.INTER_LINEAR)
        mask = cv2.imread(os.path.join(self.mask_dir, self.mask_file_list[index]))
        mask = cv2.resize(mask, dsize=(IMG_SIZE,IMG_SIZE), interpolation=cv2.INTER_NEAREST) #mask는 바이너리 형태로 되어있어야하기때문에 선형보간x

        # 마스크를 이진화 해줘야함
        mask[mask>=240] = 1
        mask[mask<240] = 0
        mask = mask/255.

        H,W,C = mask.shape
        background = np.ones((H,W,1)) #모든 채널이 label인 부분이 1, 아닌 부분이 0으로 설정 되어있음(이진화를 통해)
        background[mask[:,:,0] != 0] = 0 #각 채널이 label로 인식하고 있는 부분을 0으로 제외하기 위함
        background[mask[:,:,1] != 0] = 0
        background[mask[:,:,2] != 0] = 0

        mask = np.concatenate([background,mask], axis=-1)
        mask = np.argmax(mask, axis=-1, keepdims=False) #keepdims=True일 경우 argmax를 하더라도 채널이 압축되지 않음

        if self.transformer:
            img = self.transformer(img)
        
        #long() : classification을 pixel단위로 수행할것이기 때문에 integer로 변경(텐서로 변경하면 1. 과 같은 소수 형태로 나타나서, label은 정수임)
        mask = torch.from_numpy(mask).long()

        return img, mask
                         
        
        
        

In [8]:
data_dir = '../../data/MEDICAL-DATASET/Segmentation'
data_set = MY_Dataset(data_dir, 'train')
data_set[0] 

(array([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        ...,
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]], dtype=uint8),
 tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 

In [14]:
#transformer 
from torchvision import transforms

def build_transformer():
    transformer = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485,0.485,0.485],std=[0.229,0.224,0.225])])

    return transformer

In [32]:
# DataLoader
from torch.utils.data import DataLoader

def build_dataloader(data_dir, batch_size):
    transformer = build_transformer()

    data_loaders = {}

    train_dataset = MY_Dataset(data_dir, phase='train', transformer=transformer)
    
    # 16000장의 데이터를 불러오는 데이터 로더 중 120장만 사용할 것이기 때문에 shuffle을 false로 -> True일 경우에 에폭마다 변경
    data_loaders['train'] = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

    val_dataset = MY_Dataset(data_dir, phase='val', transformer=transformer)
    data_loaders['val'] = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    return data_loaders

In [33]:
# 구성 확인
data_dir = '../../data/MEDICAL-DATASET/Segmentation'
data_loaders = build_dataloader(data_dir, 6)

for phase in ['train','val']:
    for index, batch in enumerate(data_loaders[phase]):
        img = batch[0]
        mask = batch[1]

        print(f'{phase} img shape : {img.shape}')
        print(f'{phase} mask shape : {mask.shape}')

        if index == 0:
            break

train img shape : torch.Size([6, 3, 224, 224])
train mask shape : torch.Size([6, 224, 224])
val img shape : torch.Size([6, 3, 224, 224])
val mask shape : torch.Size([6, 224, 224])
